You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

# Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Outline:
I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's technological landscape

II. Latest Trends in Artificial Intelligence
- Machine learning advancements
- Natural language processing developments
- AI in healthcare and autonomous vehicles

III. Key Players in the AI Industry
- Google DeepMind
- IBM Watson
- Microsoft Azure AI
- Amazon Web Services AI

IV. Noteworthy News in

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

## Introduction
Artificial Intelligence (AI) continues to advance rapidly, shaping industries and revolutionizing processes that were once exclusive to human intelligence. In today's digital age, AI stands as a driving force behind innovation, offering solutions that enhance efficiency and transform decision-making across various sectors.

## Latest Trends in Artificial Intelligence
The latest trends in AI are centered around machine learning advancements, empowering systems to learn from data and enhance performance without explicit programming. Progress in natural language processing has paved the way for seamless human-machine interactions, giving rise to virtual assistants and chatbots. Furthermore, AI is making significant strides in healthcare by analyzing medical images, predicting patient outcomes, and personalizing treatment plans. In the realm of autonomous vehicles, AI is reshaping transportation by enabling self-driving cars to navigate roads safely and efficiently.

## Key Players in the AI Industry
Leading the charge in the AI industry are key players such as Google DeepMind, renowned for its breakthroughs in deep learning and reinforcement learning. IBM Watson is another prominent figure, providing cognitive computing solutions that harness AI to analyze vast amounts of data and extract valuable insights. Microsoft Azure AI offers a comprehensive suite of AI services, spanning natural language processing to computer vision. Amazon Web Services AI also plays a significant role, delivering scalable and cost-effective AI solutions for businesses of all sizes.

## Noteworthy News in Artificial Intelligence
Recent advancements in AI ethics and bias have sparked discussions on the responsible use of AI technology and the significance of addressing ethical considerations. AI applications in climate change research are showing promise in analyzing environmental data and predicting future trends to combat the impact of climate change. Additionally, the implications of AI on job automation and workforce evolution have raised concerns about the future of work, emphasizing the necessity for upskilling and reskilling to adapt to a shifting labor market.

## Target Audience Analysis
The target audience for AI-related content comprises tech-savvy individuals seeking to stay abreast of the latest advancements in the field. Business professionals aiming to integrate AI solutions into their organizations can glean insights into key trends and developments. Furthermore, students and researchers studying AI technologies can acquire valuable knowledge from in-depth analyses of industry news and advancements.

## SEO Keywords
Artificial Intelligence trends, Key players in AI industry, Latest news in AI, Future of AI technology

## Resources
1. "The Rise of Artificial Intelligence: Key Trends to Watch" - Forbes
2. "Artificial Intelligence: The Future of Technology" - Harvard Business Review
3. "AI Superpowers: China, Silicon Valley, and the New World Order" - Book by Kai-Fu Lee

**Call to Action:** Stay informed about the latest trends and advancements in Artificial Intelligence by subscribing to our newsletter for regular updates and insights.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [15]:
topic = "Become the googler"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Become the googler.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Title: Become the Googler: A Comprehensive Guide to Mastering Online Search

Introduction:
- Brief overview of the importance of effective online search skills in today's digital age.
- Introduce the concept of becoming a "googler" and the benefits it can bring to individuals.

Key Points:
1. Latest Trends:
- Importance of staying up-to-date with search engine algorithms and trends.
- Incorporating voice search optimization and mobile search trends.
- Utilizing featured snippets and knowledge graphs for enhanced 

In [16]:
Markdown(result)

# Become the Googler: A Comprehensive Guide to Mastering Online Search

In today's digital age, effective online search skills are more important than ever. Whether you're a digital marketer, SEO professional, small business owner, or blogger, mastering the art of online search can greatly benefit you. In this comprehensive guide, we will explore the concept of becoming a "googler" and the valuable insights it can bring to individuals looking to improve their online search skills.

## Importance of Staying Updated with Search Trends

One of the key aspects of becoming a successful "googler" is staying up-to-date with the latest search engine algorithms and trends. This includes incorporating voice search optimization and mobile search trends into your strategy. By utilizing featured snippets and knowledge graphs, you can enhance your search visibility and stay ahead of the competition.

## Key Players in the Search Industry

It's essential to have an overview of major search engines such as Google, Bing, and Yahoo. Additionally, understanding the key players in the SEO industry and their contributions to online search optimization can provide valuable insights into improving your search rankings. By learning from industry experts and implementing their strategies, you can increase your online visibility and drive organic traffic to your website.

## Noteworthy News and Updates in Online Search

Recent updates in search engine algorithms can have a significant impact on search results. By studying successful search optimization strategies implemented by industry leaders and keeping an eye on emerging technologies in the search industry, you can adapt your online search practices for improved results. Case studies and real-life examples can provide inspiration and guidance for optimizing your own online presence.

By following the tips for mastering online search outlined in this guide and implementing the learned techniques and tools, you can enhance your online search skills and achieve greater success in the digital landscape. It's time to become the "googler" you aspire to be and take your online search capabilities to the next level.

Remember, the key to success lies in continuous learning, adaptation, and implementation of best practices in online search optimization. Start your journey towards becoming a master of online search today!

---

This blog post has been crafted with the aim of providing valuable insights and practical tips for individuals looking to improve their online search skills and become successful "googlers." By following the outlined strategies and recommendations, you can enhance your search visibility, drive organic traffic to your website, and stay ahead of the competition in the dynamic digital landscape.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).

In [17]:
topic = "what is the feature about the AI Agent"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on what is the feature about the AI Agent.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now have all the necessary information to create a comprehensive content plan for the blog article on what is the feature about the AI Agent.

Final Answer: 

Content Plan: What is the Feature About the AI Agent

Introduction:
- Brief overview of AI technology and its impact on various industries
- Introduction to AI Agents and their role in enhancing efficiency and productivity

Key Points:
1. Latest Trends in AI Agent Features:
- Natural Language Processing capabilities for improved communication
- Machine learning algorithms for predi